# 实战 MNIST 手写数字识别

**手动计算准确度**

In [1]:
import  tensorflow as tf
from    tensorflow import keras
from    tensorflow.keras import layers, optimizers, datasets, metrics

### 1. 全链路层
**准备数据**

In [2]:
(x, y), (x_val, y_val) = datasets.mnist.load_data()
# x y x_val y_val is numpy data type
x = tf.convert_to_tensor(x, dtype=tf.float32) / 255.
y = tf.convert_to_tensor(y, dtype=tf.int32)
print(x.shape, y.shape)

(60000, 28, 28) (60000,)


In [3]:
train_dataset = tf.data.Dataset.from_tensor_slices((x, y))
train_dataset = train_dataset.batch(200)

**建立模型(未编译)**
- 不能使用 evaluate predict 方法

In [4]:
model = keras.Sequential([
    layers.Dense(256, activation='relu'),
    layers.Dense(10)
])
# 优化器， 设置学习速率
model.build(input_shape=(None, 28*28))
model.summary()
optimizer = optimizers.Adam(learning_rate=0.001)
acc_meter = metrics.Accuracy()
lossFun = keras.losses.CategoricalCrossentropy(from_logits=True)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  200960    
_________________________________________________________________
dense_1 (Dense)              multiple                  2570      
Total params: 203,530
Trainable params: 203,530
Non-trainable params: 0
_________________________________________________________________


**训练模型**

In [5]:
def train_epoch(epoch):
    for step, (x, y) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            # [b, 28, 28] => [b, 784]
            x = tf.reshape(x, (-1, 28*28))
            # Step1. compute output
            # [b, 784] => [b, 10]
            out = model(x)
            # Step2. compute loss
            y_onehot = tf.one_hot(y, depth=10)
#             loss = tf.reduce_sum(tf.square(out - y_onehot)) / x.shape[0]
            loss = lossFun(y_onehot, out)
        
        acc_meter.update_state(tf.argmax(out, axis=1), y)
        # Step3. optimize and update w1, w2, w3, b1, b2, b3
        grads = tape.gradient(loss, model.trainable_variables)
        # w' = w - lr * grad
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        
        if step % 200 == 0:
            print('epoch: %02d, step: %03d,  loss: %03f, acc: %03f' %(epoch, 
                            step, loss.numpy(), acc_meter.result().numpy()))
            acc_meter.reset_states()

In [6]:
for epoch in range(10):
    train_epoch(epoch)

epoch: 00, step: 000,  loss: 2.355545, acc: 0.050000
epoch: 00, step: 200,  loss: 0.219597, acc: 0.886400
epoch: 01, step: 000,  loss: 0.167487, acc: 0.936400
epoch: 01, step: 200,  loss: 0.116713, acc: 0.950150
epoch: 02, step: 000,  loss: 0.117420, acc: 0.959600
epoch: 02, step: 200,  loss: 0.077298, acc: 0.965650
epoch: 03, step: 000,  loss: 0.097923, acc: 0.969600
epoch: 03, step: 200,  loss: 0.055416, acc: 0.974650
epoch: 04, step: 000,  loss: 0.082226, acc: 0.977250
epoch: 04, step: 200,  loss: 0.043085, acc: 0.980750
epoch: 05, step: 000,  loss: 0.069736, acc: 0.981950
epoch: 05, step: 200,  loss: 0.035254, acc: 0.984675
epoch: 06, step: 000,  loss: 0.060254, acc: 0.984650
epoch: 06, step: 200,  loss: 0.028912, acc: 0.988000
epoch: 07, step: 000,  loss: 0.052839, acc: 0.988200
epoch: 07, step: 200,  loss: 0.023417, acc: 0.990225
epoch: 08, step: 000,  loss: 0.046715, acc: 0.990100
epoch: 08, step: 200,  loss: 0.019762, acc: 0.992425
epoch: 09, step: 000,  loss: 0.038453, acc: 0.

### use compile and fit function

In [9]:
import numpy as np

**测试**

In [10]:
def predict(X):
    X = tf.reshape(X, (-1, 28*28))
    y_predict = model(X)
    return tf.argmax(y_predict, axis=1)

def accuracy(X_test, y_test):
    y_pred = predict(X_test)
    return np.sum(y_pred == y_test) / len(y_test)

In [11]:
accuracy(x_val, y_val)

0.9766